In [88]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [89]:
out='../output/'
inp='../input/monthly/'
pref='fpw'

In [90]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
552,(2023/09/30) 23-24,80,0
553,Min,50,0
554,Max,160,0
555,Average,79.78623188405797101449275362,0


In [91]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/03/06) 04-05,29.00,50.0
6,(2017/03/06) 05-06,29.00,50.0
7,(2017/03/06) 06-07,65.00,0.0
8,(2017/03/06) 07-08,97.21,0.0
9,(2017/03/06) 08-09,97.21,0.0
...,...,...,...
548,(2023/09/30) 19-20,95.00,0.0
549,(2023/09/30) 20-21,80.00,0.0
550,(2023/09/30) 21-22,80.00,0.0
551,(2023/09/30) 22-23,80.00,0.0


In [92]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/03/06) 04-05,29.00,50.0,2017-03-06,4
6,(2017/03/06) 05-06,29.00,50.0,2017-03-06,5
7,(2017/03/06) 06-07,65.00,0.0,2017-03-06,6
8,(2017/03/06) 07-08,97.21,0.0,2017-03-06,7
9,(2017/03/06) 08-09,97.21,0.0,2017-03-06,8
...,...,...,...,...,...
548,(2023/09/30) 19-20,95.00,0.0,2023-09-30,19
549,(2023/09/30) 20-21,80.00,0.0,2023-09-30,20
550,(2023/09/30) 21-22,80.00,0.0,2023-09-30,21
551,(2023/09/30) 22-23,80.00,0.0,2023-09-30,22


In [93]:
dat=dat.drop(['datehour'],1)
dat

C:\Users\kabom\AppData\Local\Temp\ipykernel_27456\4059115739.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dat=dat.drop(['datehour'],1)


,price,volume,date,hour
5,29.00,50.0,2017-03-06,4
6,29.00,50.0,2017-03-06,5
7,65.00,0.0,2017-03-06,6
8,97.21,0.0,2017-03-06,7
9,97.21,0.0,2017-03-06,8
...,...,...,...,...
548,95.00,0.0,2023-09-30,19
549,80.00,0.0,2023-09-30,20
550,80.00,0.0,2023-09-30,21
551,80.00,0.0,2023-09-30,22


In [94]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0
2017-03-06,29.00,50.0,5,2017,3,6,0
2017-03-06,65.00,0.0,6,2017,3,6,0
2017-03-06,97.21,0.0,7,2017,3,6,0
2017-03-06,97.21,0.0,8,2017,3,6,0
...,...,...,...,...,...,...,...
2023-09-30,95.00,0.0,19,2023,9,30,5
2023-09-30,80.00,0.0,20,2023,9,30,5
2023-09-30,80.00,0.0,21,2023,9,30,5


In [95]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume    value  price_weight_av  price_max  price_min  \
year month day                                                           
2017 3     6     200.0   5800.0             29.0      97.21       29.0   
           7     400.0  11600.0             29.0      97.21       29.0   
           8     400.0  11600.0             29.0      97.21       29.0   
           9     400.0  11600.0             29.0      97.21       29.0   
           10    200.0   5800.0             29.0      97.21       29.0   
...                ...      ...              ...        ...        ...   
2023 9     26      0.0      0.0              NaN     110.00       80.0   
           27      0.0      0.0              NaN     110.00       80.0   
           28      0.0      0.0              NaN     110.00       80.0   
           29      0.0      0.0              NaN     110.00       80.0   
           30      0.0      0.0              NaN      95.00       80.0   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2017 3     6     65.852500       50.0        0.0   10.000000 2017-03-06  
           7     59.710417       50.0        0.0   16.666667 2017-03-07  
           8     59.710417       50.0        0.0   16.666667 2017-03-08  
           9     59.710417       50.0        0.0   16.666667 2017-03-09  
           10    65.852500       50.0        0.0   10.000000 2017-03-10  
...                    ...        ...        ...         ...        ...  
2023 9     26    93.125000        0.0        0.0    0.000000 2023-09-26  
           27    95.750000        0.0        0.0    0.000000 2023-09-27  
           28    93.125000        0.0        0.0    0.000000 2023-09-28  
           29    95.750000        0.0        0.0    0.000000 2023-09-29  
           30    84.375000        0.0        0.0    0.000000 2023-09-30  

[2331 rows x 10 columns]

In [96]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0,1450.0,off
2017-03-06,29.00,50.0,5,2017,3,6,0,1450.0,standard
2017-03-06,65.00,0.0,6,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,7,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,8,2017,3,6,0,0.0,morning
...,...,...,...,...,...,...,...,...,...
2023-09-30,95.00,0.0,19,2023,9,30,5,0.0,standard
2023-09-30,80.00,0.0,20,2023,9,30,5,0.0,off
2023-09-30,80.00,0.0,21,2023,9,30,5,0.0,off


In [97]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume    value  price_weight_av  price_max  price_min  \
year month day                                                           
2017 3     6     200.0   5800.0             29.0      97.21       29.0   
           7     400.0  11600.0             29.0      97.21       29.0   
           8     400.0  11600.0             29.0      97.21       29.0   
           9     400.0  11600.0             29.0      97.21       29.0   
           10    200.0   5800.0             29.0      97.21       29.0   
...                ...      ...              ...        ...        ...   
2023 9     26      0.0      0.0              NaN     110.00       80.0   
           27      0.0      0.0              NaN     110.00       80.0   
           28      0.0      0.0              NaN     110.00       80.0   
           29      0.0      0.0              NaN     110.00       80.0   
           30      0.0      0.0              NaN      95.00       80.0   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2017 3     6     65.852500       50.0        0.0   10.000000 2017-03-06  ...   
           7     59.710417       50.0        0.0   16.666667 2017-03-07  ...   
           8     59.710417       50.0        0.0   16.666667 2017-03-08  ...   
           9     59.710417       50.0        0.0   16.666667 2017-03-09  ...   
           10    65.852500       50.0        0.0   10.000000 2017-03-10  ...   
...                    ...        ...        ...         ...        ...  ...   
2023 9     26    93.125000        0.0        0.0    0.000000 2023-09-26  ...   
           27    95.750000        0.0        0.0    0.000000 2023-09-27  ...   
           28    93.125000        0.0        0.0    0.000000 2023-09-28  ...   
           29    95.750000        0.0        0.0    0.000000 2023-09-29  ...   
           30    84.375000        0.0        0.0    0.000000 2023-09-30  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 3     6                   3.0                 2.0       150.0   
           7                   3.0                 2.0       350.0   
           8                   3.0                 2.0       350.0   
           9                   3.0                 2.0       350.0   
           10                  3.0                 2.0       150.0   
...                            ...                 ...         ...   
2023 9     26                  3.0                 2.0         0.0   
           27                  3.0                 2.0         0.0   
           28                  3.0                 2.0         0.0   
           29                  3.0                 2.0         0.0   
           30                  NaN                 NaN         0.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 3     6               50.0             0.0             0.0     4350.0   
           7               50.0             0.0             0.0    10150.0   
           8               50.0             0.0             0.0    10150.0   
           9               50.0             0.0             0.0    10150.0   
           10              50.0             0.0             0.0     4350.0   
...                         ...             ...             ...        ...   
2023 9     26               0.0             0.0             0.0        0.0   
           27               0.0             0.0             0.0        0.0   
           28               0.0             0.0             0.0        0.0   
           29               0.0             0.0             0.0        0.0   
           30               0.0             NaN             NaN        0.0   

                standard_value  morning_value  evening_value  
year month day         

In [98]:
daily.to_csv(out + pref + '_daily.csv')

In [99]:
hourly.to_csv(out + pref + '_hourly.csv')

In [100]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-03-06,4,29.00,50.0,off
2017-03-06,5,29.00,50.0,standard
2017-03-06,6,65.00,0.0,morning
2017-03-06,7,97.21,0.0,morning
2017-03-06,8,97.21,0.0,morning
...,...,...,...,...
2023-09-30,19,95.00,0.0,standard
2023-09-30,20,80.00,0.0,off
2023-09-30,21,80.00,0.0,off


In [101]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [102]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_27456\3018208690.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)


,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.905773,27.494558,34.148967,64.340852,86.158317,41.674641,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,959.395238,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.272995,26.819791,47.257466,79.193597,88.723370,41.441463,50.655556,34.235088,60.633333
3,2017,6,461.038462,22765.595385,71.232500,29.95,58.551673,28.461538,10.423077,20.060897,...,40.0,49.378951,34.245704,61.625498,86.514416,115.321589,23.627240,11.700000,19.868778,7.550000
4,2017,7,457.516129,20376.515968,70.355000,25.05,52.981129,22.838710,11.580645,20.166263,...,42.0,44.537263,32.101277,59.895533,79.061769,110.973696,24.598870,10.317460,18.755102,5.476190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2023,5,418.064516,22522.580645,95.500000,52.50,80.688172,40.000000,6.451613,18.215054,...,46.0,53.873457,53.719512,54.687500,57.500000,57.500000,34.682779,1.739130,4.961240,1.739130
75,2023,6,496.000000,31981.106667,94.500000,40.00,74.123461,38.333333,5.333333,21.561111,...,44.0,64.478038,49.003276,80.382517,126.250000,126.250000,33.895706,18.181818,5.766129,27.272727
76,2023,7,481.593333,23337.378600,88.988500,29.98,62.850333,34.333333,6.730000,20.950889,...,40.0,48.458683,38.842689,59.786550,107.383012,147.970000,29.653295,9.325000,13.784255,7.500000
77,2023,8,190.645161,5957.303226,69.800000,26.82,54.869355,11.290323,2.580645,7.943548,...,46.0,31.248122,27.849512,38.771024,40.890000,NaN,12.386707,2.173913,6.434109,0.000000


In [103]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.905773,27.494558,34.148967,64.340852,86.158317,41.674641,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,959.395238,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.272995,26.819791,47.257466,79.193597,88.723370,41.441463,50.655556,34.235088,60.633333
3,2017,6,461.038462,22765.595385,71.232500,29.95,58.551673,28.461538,10.423077,20.060897,...,40.0,49.378951,34.245704,61.625498,86.514416,115.321589,23.627240,11.700000,19.868778,7.550000
4,2017,7,457.516129,20376.515968,70.355000,25.05,52.981129,22.838710,11.580645,20.166263,...,42.0,44.537263,32.101277,59.895533,79.061769,110.973696,24.598870,10.317460,18.755102,5.476190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2023,5,418.064516,22522.580645,95.500000,52.50,80.688172,40.000000,6.451613,18.215054,...,46.0,53.873457,53.719512,54.687500,57.500000,57.500000,34.682779,1.739130,4.961240,1.739130
75,2023,6,496.000000,31981.106667,94.500000,40.00,74.123461,38.333333,5.333333,21.561111,...,44.0,64.478038,49.003276,80.382517,126.250000,126.250000,33.895706,18.181818,5.766129,27.272727
76,2023,7,481.593333,23337.378600,88.988500,29.98,62.850333,34.333333,6.730000,20.950889,...,40.0,48.458683,38.842689,59.786550,107.383012,147.970000,29.653295,9.325000,13.784255,7.500000
77,2023,8,190.645161,5957.303226,69.800000,26.82,54.869355,11.290323,2.580645,7.943548,...,46.0,31.248122,27.849512,38.771024,40.890000,NaN,12.386707,2.173913,6.434109,0.000000


In [104]:
monthly=monthly.set_index(['year','month'])

In [105]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

#monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
#monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
#monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
#monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [106]:
monthly.to_csv(out + pref + '_monthly.csv')

In [107]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [108]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_27456\2677402245.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)


,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,14201.031962,47.807835,123.375591,38.950860,71.822905,23.260215,4.966129,13.054978,46.731881,...,77.000000,440231.990833,341.166667,252.000000,66.500000,44.333333,19.270622,8.811967,3.898153,1.694128
1,2,11440.262142,55.576643,136.552160,42.780556,82.598209,18.564815,2.722222,9.298200,53.121878,...,0.000000,308887.077833,306.500000,219.666667,57.500000,38.333333,15.135375,4.587412,1.146759,0.000000
2,3,11418.003532,48.097012,132.599024,42.656976,77.214578,21.496585,3.682927,10.949752,50.868811,...,71.714286,334384.389143,320.142857,240.857143,63.857143,42.571429,20.577517,4.734682,2.352601,1.893582
3,4,15729.211033,45.555353,113.969293,40.704565,67.742643,24.264130,5.753804,14.099724,46.611329,...,184.200000,413453.547143,293.857143,212.714286,55.714286,37.142857,24.776155,12.234003,11.237446,10.976407
4,5,18230.958304,42.300139,103.631304,34.573961,60.750874,31.270531,8.247343,18.167721,40.581446,...,441.542857,539115.481286,327.571429,240.428571,63.428571,42.285714,25.920344,13.359006,12.396574,12.777382
5,6,19867.912311,46.348317,126.413058,35.947816,70.972425,35.900971,6.501456,19.384790,44.141913,...,216.428571,584684.276571,323.714286,241.857143,63.857143,42.571429,30.506740,10.053983,5.921686,5.018893
6,7,17302.947046,45.413216,141.078843,37.233287,76.995884,29.861574,5.623611,16.072350,46.488836,...,165.285714,533919.508857,343.428571,249.142857,65.142857,43.428571,26.603155,7.705981,4.597991,3.889456
7,8,17089.729060,51.710607,141.470599,37.732535,74.943224,25.004608,5.856221,14.864850,45.551521,...,266.428571,529781.600857,337.571429,254.285714,67.285714,44.857143,21.199362,10.209898,6.972708,5.791925
8,9,20475.293591,49.104764,122.648325,36.168867,68.681983,33.042365,6.892611,18.724955,43.593892,...,273.571429,593783.514143,324.285714,235.714286,61.714286,41.142857,26.449418,11.533973,7.956638,6.772619
9,10,16180.074231,47.737957,109.032742,35.598925,66.964172,21.795699,7.056989,14.715757,43.896249,...,178.333333,501582.301167,342.500000,251.500000,66.000000,44.000000,19.890074,11.775993,6.521410,4.021739


In [109]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [110]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_27456\1619064361.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)


price     volume        value
month hour                                    
1     0      39.241905  20.938889   876.002468
      1      39.241905  20.938889   876.002468
      2      39.241905  20.938889   876.002468
      3      39.241905  20.938889   876.002468
      4      38.950860  20.847849   866.210317
...                ...        ...          ...
12    19    134.748883  11.788827   661.823631
      20     72.821285  20.551397  1124.578391
      21     72.821285  20.551397  1124.578391
      22     41.726872  39.601676  1652.008235
      23     41.726872  39.601676  1652.008235

[288 rows x 3 columns]

In [111]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [112]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_27456\4270116260.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)


price  volume        value
year month hour                                 
2017 3     0      27.422941    50.0  1323.629412
           1      27.422941    50.0  1323.629412
           2      27.422941    50.0  1323.629412
           3      27.422941    50.0  1323.629412
           4      27.588947    50.0  1336.931579
...                     ...     ...          ...
2023 9     19    108.695652     0.0     0.000000
           20     77.478261     0.0     0.000000
           21     77.478261     0.0     0.000000
           22     64.173913     0.0     0.000000
           23     64.173913     0.0     0.000000

[1896 rows x 3 columns]

In [113]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [114]:
from statsmodels.tsa.stattools import pacf

In [115]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.735036,0.780486,0.782111,0.740840
2,-0.055865,-0.182452,-0.171190,-0.161691
3,0.230746,0.285129,0.281513,0.264728
4,0.177651,0.151214,0.172873,0.082159
5,0.030179,0.115539,0.091647,0.085670
6,0.082505,0.034526,0.086625,0.004956
7,0.079398,-0.139671,-0.136195,0.190997
8,-0.061808,0.004082,0.002102,-0.041132
9,-0.268637,-0.290523,-0.307269,-0.134138


In [116]:
pacfs.to_csv(out + pref + '_pacfs.csv')